In [ ]:
pip install prophet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from google.colab import drive
from statsmodels.tsa.arima.model import ARIMA
warnings.filterwarnings("ignore")
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
sensor_data_path = '/content/drive/MyDrive/city_day.csv'
# Load the dataset
df = pd.read_csv(sensor_data_path)

In [ ]:
df.head()

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,2015-01-01,NaN,NaN,0.92,18.22,17.15,NaN,0.92,27.64,133.36,0.00,0.02,0.00,NaN,NaN
1,Ahmedabad,2015-01-02,NaN,NaN,0.97,15.69,16.46,NaN,0.97,24.55,34.06,3.68,5.50,3.77,NaN,NaN
2,Ahmedabad,2015-01-03,NaN,NaN,17.40,19.30,29.70,NaN,17.40,29.07,30.70,6.80,16.40,2.25,NaN,NaN
3,Ahmedabad,2015-01-04,NaN,NaN,1.70,18.48,17.97,NaN,1.70,18.59,36.08,4.43,10.14,1.00,NaN,NaN
4,Ahmedabad,2015-01-05,NaN,NaN,22.10,21.42,37.76,NaN,22.10,39.33,39.31,7.01,18.89,2.78,NaN,NaN


In [ ]:
pmean=df["PM2.5"].mean()
df["PM2.5"].fillna(pmean,inplace=True)
pmmean=df["PM10"].mean()
df["PM10"].fillna(pmmean,inplace=True)
nmean=df["NO"].mean()
df["NO"].fillna(nmean,inplace=True)
nomean=df["NO2"].mean()
df["NO2"].fillna(nomean,inplace=True)
noxmean=df["NOx"].mean()
df["NOx"].fillna(noxmean,inplace=True)
nhmean=df["NH3"].mean()
df["NH3"].fillna(nhmean,inplace=True)
cmean=df["CO"].mean()
df["CO"].fillna(cmean,inplace=True)
smean=df["SO2"].mean()
df["SO2"].fillna(smean,inplace=True)
omean=df["O3"].mean()
df["O3"].fillna(omean, inplace=True)
bmean=df["Benzene"].mean()
df["Benzene"].fillna(bmean,inplace=True)
tmean=df["Toluene"].mean()
df["Toluene"].fillna(tmean,inplace=True)
xmean=df["Xylene"].mean()
df["Xylene"].fillna(xmean,inplace=True)
amean=df["AQI"].mean()
df["AQI"].fillna(amean,inplace=True)
df=df.drop('AQI_Bucket',axis=1)
df.isna().sum()

City       0
Date       0
PM2.5      0
PM10       0
NO         0
NO2        0
NOx        0
NH3        0
CO         0
SO2        0
O3         0
Benzene    0
Toluene    0
Xylene     0
AQI        0
dtype: int64

In [ ]:
from prophet import Prophet

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df = df[['Date', 'PM2.5']]
df = df.rename(columns={'Date': 'ds', 'PM2.5': 'y'})

In [ ]:
model = Prophet()
model.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpr_cznhja/t_k1hwlt.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpr_cznhja/j4tb8i_r.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=82089', 'data', 'file=/tmp/tmpr_cznhja/t_k1hwlt.json', 'init=/tmp/tmpr_cznhja/j4tb8i_r.json', 'output', 'file=/tmp/tmpr_cznhja/prophet_model5mjek16b/prophet_model-20230702063647.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
06:36:47 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
06:37:04 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
future = model.make_future_dataframe(periods=30,freq='D', include_history=False )  # Dự báo cho 30 ngày tiếp theo
forecast = model.predict(future)
forecast.rename(columns={'yhat': 'pm25'}, inplace=True)
forecast_30days = forecast[['ds', 'pm25']]
forecast_30days

,ds,pm25
0,2020-07-02,15.870192
1,2020-07-03,14.953989
2,2020-07-04,13.794113
3,2020-07-05,13.331202
4,2020-07-06,13.080026
5,2020-07-07,13.780749
6,2020-07-08,13.941713
7,2020-07-09,13.095039
8,2020-07-10,12.378360
9,2020-07-11,11.394183


In [ ]:
from prophet import Prophet
import joblib

# model.save_model("prophet_model.h5")

# Lưu mô hình
joblib.dump(model, "prophet_model.joblib")

['prophet_model.joblib']

In [ ]:
loaded_model = joblib.load("prophet_model.joblib")

In [ ]:
futureload = loaded_model.make_future_dataframe(periods=30, freq='D', include_history=False)  # Dự báo cho 30 ngày tiếp theo
forecast2 = loaded_model.predict(futureload)
forecast2.rename(columns={'yhat': 'pm25'}, inplace=True)
forecast_30days = forecast2[['ds', 'pm25']]
forecast_30days

,ds,pm25
0,2020-07-02,15.870192
1,2020-07-03,14.953989
2,2020-07-04,13.794113
3,2020-07-05,13.331202
4,2020-07-06,13.080026
5,2020-07-07,13.780749
6,2020-07-08,13.941713
7,2020-07-09,13.095039
8,2020-07-10,12.378360
9,2020-07-11,11.394183


In [ ]:
forecast_30days['pm25'].to_csv('/content/drive/MyDrive/predicted_prophetpm2_5.csv', index=False)

In [ ]:
!pip install firebase-admin

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
json_path = './content/drive/MyDrive/projectnt114-firebase-adminsdk-3z33d-6d4b587f61'
cred = credentials.Certificate('/content/sample_data/serviceAccountkey.json')

firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://projectnt114-default-rtdb.firebaseio.com'
})

In [ ]:
import json
import pandas as pd
pf = pd.read_csv('/content/drive/MyDrive/predicted_prophetpm2_5.csv')
json_data = pf.to_dict(orient='records')

ref = db.reference('/Prophet')

for row in json_data:
    for key, value in row.items():
        column_ref = ref.child(key.replace(".", "_")).push()
        column_ref.set(value)